## What should I expect the data format to be?
Each sample in the train and test set has the following information:

The text of a tweet
A keyword from that tweet (although this may be blank!)
The location the tweet was sent from (may also be blank)
## What am I predicting?
You are predicting whether a given tweet is about a real disaster or not. If so, predict a 1. If not, predict a 0.

In [7]:
import pandas as pd
import polars as pl


In [24]:
train_pl = pl.read_csv('./train.csv')
# train_pd = pd.read_csv('./train.csv')
test_pl = pl.read_csv('./test.csv')
# test_pd = pd.read_csv('./test.csv')


In [25]:
train_pl.filter(pl.col('target')==1).head()


id,keyword,location,text,target
i64,str,str,str,i64
1,null,null,"""Our Deeds are …",1
4,null,null,"""Forest fire ne…",1
5,null,null,"""All residents …",1
6,null,null,"""13,000 people …",1
7,null,null,"""Just got sent …",1


In [26]:
train_pl.filter(pl.col('target')==0).head()


id,keyword,location,text,target
i64,str,str,str,i64
23,null,null,"""What's up man?…",0
24,null,null,"""I love fruits""",0
25,null,null,"""Summer is love…",0
26,null,null,"""My car is so f…",0
28,null,null,"""What a goooooo…",0


In [20]:
train_pl.describe()


describe,id,keyword,location,text,target
str,f64,str,str,str,f64
"""count""",7613.0,"""7613""","""7613""","""7613""",7613.0
"""null_count""",0.0,"""61""","""2533""","""0""",0.0
"""mean""",5441.934848,null,null,null,0.42966
"""std""",3137.11609,null,null,null,0.49506
"""min""",1.0,"""ablaze""",""" ""","""! Residents Re…",0.0
"""25%""",2734.0,null,null,null,0.0
"""50%""",5408.0,null,null,null,0.0
"""75%""",8146.0,null,null,null,1.0
"""max""",10873.0,"""wrecked""","""åø\_(?)_/åø""","""åÈMGN-AFRICAå¨…",1.0


In [31]:
train_pl.filter(pl.col('location').is_not_null())


id,keyword,location,text,target
i64,str,str,str,i64
48,"""ablaze""","""Birmingham""","""@bbcmtd Wholes…",1
49,"""ablaze""","""Est. September…","""We always try …",0
50,"""ablaze""","""AFRICA""","""#AFRICANBAZE: …",1
52,"""ablaze""","""Philadelphia, …","""Crying out for…",0
53,"""ablaze""","""London, UK""","""On plus side L…",0
54,"""ablaze""","""Pretoria""","""@PhDSquares #m…",0
55,"""ablaze""","""World Wide!!""","""INEC Office in…",1
57,"""ablaze""","""Paranaque City…","""Ablaze for you…",0
59,"""ablaze""","""Live On Webcam…","""Check these ou…",0


In [32]:
train_pl.filter(pl.col('keyword').is_not_null())


id,keyword,location,text,target
i64,str,str,str,i64
48,"""ablaze""","""Birmingham""","""@bbcmtd Wholes…",1
49,"""ablaze""","""Est. September…","""We always try …",0
50,"""ablaze""","""AFRICA""","""#AFRICANBAZE: …",1
52,"""ablaze""","""Philadelphia, …","""Crying out for…",0
53,"""ablaze""","""London, UK""","""On plus side L…",0
54,"""ablaze""","""Pretoria""","""@PhDSquares #m…",0
55,"""ablaze""","""World Wide!!""","""INEC Office in…",1
56,"""ablaze""",null,"""Barbados #Brid…",1
57,"""ablaze""","""Paranaque City…","""Ablaze for you…",0


In [ ]:
X_train = train_pl.drop(columns='target')
y_train = train_pl['target']


In [33]:
max_len = X_train['text'].str.lengths().max()
pl.Config. set_fmt_str_lengths(max_len)


/tmp/ipykernel_4472/2668503639.py:1: DeprecationWarning: `lengths` is deprecated. It has been renamed to `len_bytes`.
  max_len = X_train['text'].str.lengths().max()


polars.config.Config

In [34]:
X_train.head()


id,keyword,location,text
i64,str,str,str
1,null,null,"""Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all"""
4,null,null,"""Forest fire near La Ronge Sask. Canada"""
5,null,null,"""All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"""
6,null,null,"""13,000 people receive #wildfires evacuation orders in California """
7,null,null,"""Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school """


In [35]:
X_train.filter(pl.col('keyword').is_not_null()).head()


id,keyword,location,text
i64,str,str,str
48,"""ablaze""","""Birmingham""","""@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C"""
49,"""ablaze""","""Est. September 2012 - Bristol""","""We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw"""
50,"""ablaze""","""AFRICA""","""#AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi"""
52,"""ablaze""","""Philadelphia, PA""","""Crying out for more! Set me ablaze"""
53,"""ablaze""","""London, UK""","""On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N"""


In [36]:
X_train['keyword'].unique()


keyword
str
"""blaze"""
"""volcano"""
"""derailment"""
"""trapped"""
"""tornado"""
"""devastated"""
"""chemical%20emergency"""
"""destroyed"""
"""structural%20failure"""


In [37]:
X_train['location'].unique()


location
str
"""Sunny South florida """
"""Freddy Fazbears pizzeria"""
"""Bulgaria"""
"""Van Buren, MO"""
"""At your back"""
"""Huntsville AL"""
"""Peterborough, Ont."""
"""Florida but I wanna be n Texas"""
"""Essex/Brighton"""
